In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs1999.db")

In [3]:
df = pd.read_csv("1999 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1999", db, if_exists="replace")

os.getcwd()



C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1999_index,monarchs1999,3,"CREATE INDEX ""ix_monarchs1999_index""ON ""monarc..."
1,table,monarchs1999,monarchs1999,2,"CREATE TABLE ""monarchs1999"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1999")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,"Tag City, State",TEXT,0,None,0
4,4,Tag Date,TEXT,0,None,0
5,5,Report Date,TEXT,0,None,0
6,6,Report Location,TEXT,0,None,0
7,7,Reporter,TEXT,0,None,0
8,8,Miles,REAL,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Reporter"], axis = 1, inplace = True)
df

,Tag No.,"Tag City, State",Tag Date,Report Date,Report Location,Miles
0,972GM,"Durham, ME",08/19/99,02/00,"El Rosario, MICH, MX",2421.0
1,097KD,"Salisbury, MA",09/12/99,01/28/00,"El Rosario, MICH, MX",2344.0
2,575NE,"Essex Junction, VT",09/20/99,03/01/00,"El Rosario, MICH, MX",2320.0
3,813GR,"Bolton Landing, NY",09/24/99,01/14/00,"El Rosario, MICH, MX",2258.0
4,117JG,"Amherst, MA",09/01/99,02/00,"El Rosario, MICH, MX",2255.0
...,...,...,...,...,...,...
746,AB106,NaN,NaN,09/22/99,"Brown Deer, WI",NaN
747,597NJ,"Oshkosh, WI",NaN,09/17/99,"Oshkosh, WI",NaN
748,034GA,"Grand Rapids, MI",NaN,08/24/99,"Pellston, MI",NaN
749,914LB,"Saratoga Springs, NY",NaN,10/15/99,"Schuylerville, NY",NaN


### Split & rename combined columns in Tagged category

In [7]:
df2 = df['Tag City, State'].str.split(', ', expand=True)
df2.columns = ['tag_city', 'Tag State']
df2.loc[df2['Tag State'] == 'ONT', 'Tag State'] = 'Ontario'
df2.loc[df2['Tag State'] == 'QUE', 'Tag State'] = 'Quebec'
df2

,tag_city,Tag State
0,Durham,ME
1,Salisbury,MA
2,Essex Junction,VT
3,Bolton Landing,NY
4,Amherst,MA
...,...,...
746,NaN,NaN
747,Oshkosh,WI
748,Grand Rapids,MI
749,Saratoga Springs,NY


### Split & Rename combined columns in Reported category

In [8]:
df3 = df['Report Location'].str.split(', ', expand=True)
df3.columns = ['report_city', 'Report State', 'Report Country']
df3.loc[df3['Report State'] == 'MICH', 'Report State'] = 'Michoacán'
df3.loc[df3['Report State'] == 'MEXICO', 'Report State'] = 'Mexico'
df3.loc[df3['Report State'] == 'ONT', 'Report State'] = 'Ontario'
df3

,report_city,Report State,Report Country
0,El Rosario,Michoacán,MX
1,El Rosario,Michoacán,MX
2,El Rosario,Michoacán,MX
3,El Rosario,Michoacán,MX
4,El Rosario,Michoacán,MX
...,...,...,...
746,Brown Deer,WI,None
747,Oshkosh,WI,None
748,Pellston,MI,None
749,Schuylerville,NY,None


### Create new table with unneeded columns dropped

In [9]:
df4 = pd.concat([df, df2, df3], axis=1)
df4.drop(['Tag City, State', 'Report Location', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,972GM,ME,08/19/99,Michoacán,MX,02/00,2421.0
1,097KD,MA,09/12/99,Michoacán,MX,01/28/00,2344.0
2,575NE,VT,09/20/99,Michoacán,MX,03/01/00,2320.0
3,813GR,NY,09/24/99,Michoacán,MX,01/14/00,2258.0
4,117JG,MA,09/01/99,Michoacán,MX,02/00,2255.0
...,...,...,...,...,...,...,...
746,AB106,NaN,NaN,WI,None,09/22/99,NaN
747,597NJ,WI,NaN,WI,None,09/17/99,NaN
748,034GA,MI,NaN,MI,None,08/24/99,NaN
749,914LB,NY,NaN,NY,None,10/15/99,NaN


### Rename values in Report Country column

In [10]:
new_table.loc[new_table['Report Country'] == 'MX', 'Report Country'] = 'MEXICO'
new_table.loc[new_table['Report Country'].isnull(), 'Report Country'] = 'USA'
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,972GM,ME,08/19/99,Michoacán,MEXICO,02/00,2421.0
1,097KD,MA,09/12/99,Michoacán,MEXICO,01/28/00,2344.0
2,575NE,VT,09/20/99,Michoacán,MEXICO,03/01/00,2320.0
3,813GR,NY,09/24/99,Michoacán,MEXICO,01/14/00,2258.0
4,117JG,MA,09/01/99,Michoacán,MEXICO,02/00,2255.0
...,...,...,...,...,...,...,...
746,AB106,NaN,NaN,WI,USA,09/22/99,NaN
747,597NJ,WI,NaN,WI,USA,09/17/99,NaN
748,034GA,MI,NaN,MI,USA,08/24/99,NaN
749,914LB,NY,NaN,NY,USA,10/15/99,NaN


In [11]:
new_table['Season']='1999'

### Change Values in Report Date where day of month is missing

In [12]:
new_table.loc[new_table['Report Date'] == '02/00', 'Report Date'] = '02/01/00'
new_table.loc[new_table['Report Date'] == '5/00', 'Report Date'] = '05/01/00'
new_table.loc[new_table['Report Date'] == '04/00', 'Report Date'] = '04/01/00'

### Final check for unwanted spaces

In [13]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [14]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                    float64
Season                    object
dtype: object

In [15]:
del df2, df3, df4, df5

### Export cleaned data to csv & pkl

In [17]:
new_table.to_pickle("Monarchs1999.pkl")
new_table.to_csv("csv/Monarchs1999.csv")